In [2]:
import json
import os
import csv

In [4]:
# for released days
def Load_datelist_train():
    with open("data_datelist_train.txt", 'r') as f:
        datelist = f.read().split()
    datelist.sort()
    return datelist

In [8]:
def Load_datelist_test():
    with open("data_datelist_test.txt", 'r') as f:
        datelist = f.read().split()
    datelist.sort()
    return datelist

In [ ]:
def Load_datelist_test():
    with open("data_datelist_test.txt", 'r') as f:
        datelist = f.read().split()
    datelist.sort()
    return datelist

In [ ]:
def Create_datelist_train():
    datelist=os.listdir("html.2023.final.data/release")
    datelist.remove(".DS_Store")
    datelist.remove("20231015") # almost empty
    datelist.sort()
    with open("data_datelist.txt", 'w') as f:
        for date in datelist:
            f.write(f"{date}\n")
    print("datelist:", datelist)
    return datelist

In [4]:
def Load_all_stns():
    # for released stations
    with open("html.2023.final.data/demographic.json", "r") as f:
        stations = json.load(f)
    stations = list(stations.keys())
    return stations

In [5]:
# load stations which we want to predict
def Load_predict_stns():
    with open("html.2023.final.data/sno_test_set.txt", "r") as f:
        predict_stns = f.read().split()
        predict_stns.sort()
    return predict_stns

In [6]:
# load time list
def Load_time_list():
    time_list = []
    for hour in range(24):
            for min in range(60):
                h = str(hour) if hour > 9 else '0' + str(hour)
                m = str(min) if min > 9 else '0' + str(min)
                time_list.append(f"{h}:{m}")
    return time_list

In [7]:
def Load_date_feature():
    # read csv data
    day_feature = {}
    csv_file_path = "data_dayfeature.csv"
    with open(csv_file_path, 'r') as csv_file:
        # Create a CSV reader
        csv_reader = csv.DictReader(csv_file)

        # Iterate over each row in the CSV file
        for row in csv_reader:
            day_feature[row["date"]] = {}
            for feature in ['weekday', 'workingday', 'holiday']:
                day_feature[row["date"]][feature] = row[feature]
    return day_feature

In [8]:
def Load_stn_tot():
    with open("./data_stn_tot.json", 'r') as f:
        stn_tot = json.load(f)
    return stn_tot

In [9]:
def Load_weather_data():
    with open("./data_weather.json", "r") as f:
        weather_data = json.load(f)
    return weather_data

In [10]:
def Load_one_weekdate_list():
    weekdate_list = ['20231002', "20231003", "20231004", "20231005", "20231006", "20231007", "20231008", "20231009"]
    return weekdate_list

In [43]:

# gen data for train
def Gen_train_data():
    predict_stns = Load_predict_stns()
    date_list = Load_datelist_train()
    time_list = Load_time_list()
    weekdate_list = Load_one_weekdate_list()
    
    print("predict_stns", predict_stns)
    print("date_list", date_list)
    print("time_list", time_list)
    print("weekdate_list", weekdate_list)
    
    
    for stn in predict_stns:    
        train_file = open(f"./data_train/{stn}_train.txt", 'w')
        with open(f"./data_by_station/{stn}.json", "r") as f:
            source_data = json.load(f)
        
        for date in date_list:
            print(date, stn)
            for time in time_list:
                train = f"{stn} {date} {time}"
                
                for feature in ["tot", "weekday", "workingday", "holiday", "rain", "temp", "moist", "sbi"]:
                    train = train + " " + str(source_data[time][date][feature])
                    
                # add day fisrt stn sbi
                # train = train + " " + str(source_data["00:00"][date]["sbi"])
                # print(train)
                train_file.write(train + "\n")
            
        train_file.close()
        
    print("complete train data gen")

In [51]:
# Gen_train_data()

datelist:  ['20231002', '20231003', '20231004', '20231005', '20231006', '20231007', '20231008', '20231009', '20231010', '20231011', '20231016', '20231017', '20231018', '20231019', '20231020', '20231025', '20231026', '20231027', '20231028', '20231029', '20231030', '20231031', '20231101', '20231102', '20231103', '20231104', '20231105', '20231106', '20231107', '20231108', '20231109', '20231110', '20231111', '20231112', '20231113', '20231114', '20231115', '20231116', '20231117', '20231118', '20231119', '20231120', '20231121', '20231122', '20231123', '20231124', '20231125', '20231126', '20231127']
predict_stns ['500101001', '500101002', '500101003', '500101004', '500101005', '500101006', '500101007', '500101008', '500101009', '500101010', '500101013', '500101014', '500101015', '500101018', '500101019', '500101020', '500101021', '500101022', '500101023', '500101024', '500101025', '500101026', '500101027', '500101028', '500101029', '500101030', '500101031', '500101032', '500101033', '50010103

In [29]:
# gen data for test
def Gen_test_data():
    testday_list = Load_datelist_test()
    predict_stns = Load_predict_stns()
    time_list = Load_time_list()
    stn_tot = Load_stn_tot()
    day_feature = Load_date_feature()
    weather_data = Load_weather_data()
    weekdate_list = Load_one_weekdate_list()
    
    print("testday_list:", testday_list)
    print("predict_stns:", predict_stns)
    print("time_list:", time_list)
    print("stn_tot:", stn_tot)
    print("day_feature:", day_feature)
    print("weather_data:", weather_data)
    print("weekdate_list", weekdate_list)
    
    for stn in predict_stns:
        print(stn)
        test_file = open(f'./data_test/{stn}_test.txt', 'w')
        
        with open(f"./data_by_station/{stn}.json", "r") as f:
            source_data = json.load(f)
            
        for date in testday_list:
            for time in time_list:
                test_data = f"{stn} {date} {time} {stn_tot[stn]}"
                
                # add day feature
                for feature in ["weekday", "workingday", "holiday"]:
                    test_data = test_data + " " + day_feature[date][feature]
                    
                # add weather feature
                for feature in ["rain","temp", "moist"]:
                    if date not in weather_data:
                        test_data = test_data + ' ' + str(0)
                    else: 
                        test_data = test_data + ' ' + str(weather_data[date][str( int( time[:2]) + 1) ][feature])
                
                # add stn's "sbi"
                test_data = test_data + " " + "0"
                # add day first stn's sbi
                # test_data = test_data + " " + "0"
                test_file.write(test_data + "\n")
        test_file.close()
        
    print("complete test data gen")

In [50]:
# Gen_test_data()

predict_stns: ['500101001', '500101002', '500101003', '500101004', '500101005', '500101006', '500101007', '500101008', '500101009', '500101010', '500101013', '500101014', '500101015', '500101018', '500101019', '500101020', '500101021', '500101022', '500101023', '500101024', '500101025', '500101026', '500101027', '500101028', '500101029', '500101030', '500101031', '500101032', '500101033', '500101034', '500101035', '500101036', '500101037', '500101038', '500101039', '500101040', '500101041', '500101042', '500101091', '500101092', '500101093', '500101094', '500101114', '500101115', '500101123', '500101166', '500101175', '500101176', '500101181', '500101184', '500101185', '500101188', '500101189', '500101190', '500101191', '500101193', '500101199', '500101209', '500101216', '500101219', '500105066', '500106002', '500106003', '500106004', '500119043', '500119044', '500119045', '500119046', '500119047', '500119048', '500119049', '500119050', '500119051', '500119052', '500119053', '500119054